In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
DATA = '../data/'
KAG = DATA + 'kaggle_data/'

teams = pd.read_csv(KAG + 'MTeams.csv')
reg = pd.read_csv(KAG + 'MRegularSeasonCompactResults.csv')
post = pd.read_csv(KAG  + 'MNCAATourneyCompactResults.csv')
cities = pd.read_csv(KAG + 'Cities.csv')
gameCities = pd.read_csv(KAG + 'MGameCities.csv')
ords = pd.read_csv(KAG + 'MMasseyOrdinals.csv')

## 1. Construct 'favorite' model

### Description: 

Set of games considered:
All regular season games

Inputs used:
Rating systems + Distance to games

### Why favored:

-Lots of data - Rating systems and results go back to 2003

-"Compact" variables - ratings systems are designed to capture a lot of what we would already be trying to model if we were to feature engineer ourselves from the raw data

In [59]:
teams = pd.read_csv(KAG + 'MTeams.csv')
reg = pd.read_csv(KAG + 'MRegularSeasonCompactResults.csv')
post = pd.read_csv(KAG  + 'MRegularSeasonCompactResults.csv')
cities = pd.read_csv(KAG + 'Cities.csv')
gameCities = pd.read_csv(KAG + 'MGameCities.csv')
ords = pd.read_csv(KAG + 'MMasseyOrdinals.csv')

In [67]:
assert all(post.columns == reg.columns)
all_games = pd.concat([reg, post])
all_games['wins'] = 1
all_games.rename(columns = {'WTeamID':'Tm1ID', 
                            'LTeamID':'Tm2ID',
                            'WScore' : 'Tm1Score', 
                            'LScore' : 'Tm2Score', 
                            'WLoc' : 'Tm1Loc'}, inplace = True)
all_games
all_games_c = all_games.copy()
all_games_c.rename(columns = {'Tm1ID' : 'Tm2ID', 
                            'Tm2ID' : 'Tm1ID',
                            'Tm1Score' : 'Tm2Score', 
                            'Tm2Score' : 'Tm1Score'
                            })

all_games_c['Tm1Loc'] = all_games_c['Tm1Loc'].replace({'H': 'A', 'A':'H'})

# all_games.groupby(['Year','WTeamID'])
# all_games[all_games['WLoc'] == 'A'].groupby(['Season','WTeamID'], as_index = False).sum()[['Season','WTeamID','wins']]
# all_games_c[all_games_c['WLoc'] == 'H'].groupby(['Season','WTeamID'], as_index = False).sum()[['Season','WTeamID','wins']]
# (all_games_c['WLoc'] == 'H').sum()
def checkFlips(all_games, all_games_c):
    a = all_games.groupby(['WTeamID'], as_index = False).sum()[['WTeamID','wins']]
    b = all_games.groupby(['LTeamID'], as_index = False).sum()[['LTeamID','wins']]
    c = all_games.groupby(['LTeamID'], as_index = False).sum()[['LTeamID','wins']]
    d = all_games.groupby(['WTeamID'], as_index = False).sum()[['WTeamID','wins']]

    e = a.merge(b, suffixes = ['_W','_L'], left_on = 'WTeamID', right_on = 'LTeamID')
    f = c.merge(d, suffixes = ['_L','_W'], left_on = 'LTeamID', right_on = 'WTeamID')
    e.merge(f, on = ['WTeamID','LTeamID','wins_W','wins_L'])
    if len(e) == len(a):
        return True
    return False

In [65]:
all_games.tail(1)

,Season,DayNum,Tm1ID,Tm1Score,Tm2ID,Tm2Score,Tm1Loc,NumOT,wins
176079,2022,132,1463,66,1343,64,N,0,1
